

## Feature Engineering


We will add some more features.

In [1]:
import warnings
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.special import inv_boxcox, boxcox1p
from pprint import pprint

# supress unnecessary warnings for readability and cleaner presentation
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('bmh')

In [3]:
# helper function which performs all steps of data processing which we applied during 
# Data cleaning and preprocessing
def read_clean_data():
    
    # reading data
    df_train = pd.read_csv('use_case_data.csv')
    df_score = pd.read_csv('score_data.csv')

    df_train['data_type'] = 'train'
    df_score['data_type'] = 'score'
    
    # removing negative market shares
    temp = df_train[df_train['Market_Share'] >= 0]
    
    # applying boxcox
    market_share = stats.boxcox(temp['Market_Share'].values + 1e-30, lmbda=0.25)
    
    # combining train and score
    data = pd.concat([temp.drop(columns='Market_Share'), df_score],ignore_index=True)
    
    # adding response variable
    data['Market_Share'] = np.nan
    mask = data['data_type'] == 'train'
    data.loc[mask, 'Market_Share'] = market_share
    data.drop(columns='data_type', inplace=True)
    
    # changing dtype to category
    data['ITEMSCODE'] = data['ITEMSCODE'].astype('category')

    # spliting to year and month, adding to not_useful_features list
    data['LAUNCH_YEAR'], data['LAUNCH_MONTH'] = data['NPLLAUNCHDATE'].map(str).apply(lambda x: [x[:4], x[4:]]).str   
    
    # removing not useful features
    not_useful = ['BRMID', 'LATESTPERIODINDEX', 'NPLLAUNCHDATE', 'ISREPLACEMENT', 'BRM', 'MARKETEDBRAND',
                  'BRANDSUBFAMILY', 'NPLLAUNCHYEAR', 'RTYPE', 'ITEMSHAPE']
    
    data.drop(columns=not_useful, inplace=True)
    
    # filling missing values
    data['SPECIALFLAVOR'].fillna('NOSPECIALFLAVOR', inplace=True)
    data['TIPCOLOR'].fillna('NOTIPCOLOR', inplace=True)
    
    # transform categorical features into the appropriate type
    for c in data.columns:
        col_type = data[c].dtype
        if col_type == 'object' or col_type.name == 'category':
            data[c] = data[c].astype('category')
            
    # transforming with boxcox1p for reducing skew
    data[['LEN', 'NCON', 'RETAILPACKPRICE']] = boxcox1p(data[['LEN', 'NCON', 'RETAILPACKPRICE']], -0.25)

    print('all data shape: {}'.format(data.shape))
    return data

We will add some more features by aggregating some categorical features by numeric ones.

For example, we will calculate **mean, median, std, skew** of **RETAILPACKPRICE** for each **REGION**. 

In [4]:
def aggregate(data, f_1, f_2):
    """Aggregates f_1 by f_2"""

    mean_ = '{}_{}_mean'.format(f_1, f_2)
    median_ = '{}_{}_median'.format(f_1, f_2)
    std_ = '{}_{}_std'.format(f_1, f_2)
    skew_ = '{}_{}_skew'.format(f_1, f_2)
    
    # aggregation of f_1 by f_2
    temp = data.groupby(f_1)[f_2].aggregate({
        mean_: np.mean,
        std_: np.std,
        median_: np.median,
        skew_: stats.skew
    }).reset_index()
    
    # filling nans with 0
    for x in (mean_, std_, median_, skew_):
        temp[x].fillna(0, inplace=True)
        
    data = data.merge(temp)
    
    # adding relative (for example product RETAILPACKPRICE / REGION mean RETAILPACKPRICE)
    relative_mean_ = '{}_{}_relative_mean'.format(f_1, f_2)
    relative_median_ = '{}_{}_relative_median'.format(f_1, f_2)
    
    def relative(row, f):
        f_value, f_2_value = row[f], row[f_2]
        if f_value == 0 and f_2_value == 0:
            return 1.
        elif f_value == 0 and f_2_value != 0:
            return -99999
        else:
            return f_2_value / f_value
        
#     data[relative_mean_] = data.apply(lambda row: relative(row, mean_), axis=1)
    data[relative_median_] = data.apply(lambda row: relative(row, median_), axis=1)
    return data

In [5]:
def feature_engineering(data, to_drop=None, to_aggregate=None, to_dummy=False, cardinality_ratio=1., ):
    """
    Recives cleaned and preprocessed data, add features, high cardinality features and makes one-hot encoding.
    """
    temp = data.copy()

    # removing high cardinality categorical features
    if cardinality_ratio < 1.:
        n = len(temp)
        high_cardinalty_features = []
        for c in temp.columns:
            if temp[c].dtype.name == 'category' and len(temp[c].unique()) / n > cardinality_ratio:
                high_cardinalty_features.append(c)
        print('Removing {} features, which have high cardinality.'.format(high_cardinalty_features))
        temp.drop(columns=high_cardinalty_features, inplace=True)
    
    # aggergation
    numeric_fs = [c for c in temp.columns if temp[c].dtype.name != 'category' and c != 'Market_Share']
    for f_1 in to_aggregate:
        for f_2 in numeric_fs:
            temp = aggregate(temp, f_1, f_2)
    
    # adding also **2, **3 and **0.5 for LEN and RETAILPACKPRICE
    for x in ['LEN', 'RETAILPACKPRICE']:
        temp['{}_**2'.format(x)] = temp[x] ** 2
        temp['{}_**3'.format(x)] = temp[x] ** 2
        temp['{}_**0.5'.format(x)] = np.sqrt(np.abs(temp[x]))        
    print('all data shape after features addition: {}'.format(temp.shape))
    
    # dropping some features
    if to_drop:
        temp.drop(columns=to_drop, inplace=True)
        print('all data shape after features deletion: {}'.format(temp.shape))
    

    # one-hot encoding for categorical features
    if to_dummy:
        temp = pd.get_dummies(temp)
        print('all data shape after one-hot-encoding: {}'.format(temp.shape))

    # splitting into train and score
    mask = temp['Market_Share'].notnull()
    training_data, score_data = temp[mask], temp[np.invert(mask)]
    score_data.drop(columns='Market_Share', inplace=True)
    print('training data shape: {}, score data shape: {}'.format(
        training_data.shape, score_data.shape))
    return training_data, score_data

**Preparing data: data clening, preprocessing and feature engineering, one-hot-encoding**

In [6]:
df = read_clean_data()

all data shape: (1721, 32)


In [7]:
# categorical features, which will be aggregated
to_aggregate = [
    'TCLASS',
    'REGION',
    'LOCALCLASS',
    'BLDIMAGE', 
    'MARKET',
    'MINDICATOR', 
    'THICATEGORY', 
    'PCKT',
]

# categorical features, which will be dropped
to_drop = [
    'BRANDSUBFAMILYGROUPING', 
    'BRANDSUBFAMILYLINE',
    'BRANDSUBFAMILYGROUP', 
    'BRANDONMARKET',
    'BRANDDIFFERENTIATOR',
]

training_df, score_df = feature_engineering(df, 
                                            to_drop=to_drop, 
                                            to_aggregate=to_aggregate,
                                            to_dummy=True, 
                                            cardinality_ratio=1.)

numeric_fs = [c for c in training_df.columns if training_df[c].dtype.name != 'category' and c != 'Market_Share']
categoircal_fs = list(set(training_df.columns) - set(numeric_fs) - {'Market_Share'})

all data shape after features addition: (1721, 198)
all data shape after features deletion: (1721, 193)
all data shape after one-hot-encoding: (1721, 517)
training data shape: (1692, 517), score data shape: (29, 516)


## Evaluation metrics

In [8]:
import itertools
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix, make_scorer
from sklearn.base import clone
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from tabulate import tabulate

In [9]:
def plot_confusion_matrix(matrix, classes, cmap=plt.cm.Reds):
    """This function plots the normalized confusion matrix."""
    matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title('Normalized Confusion matrix', fontsize=17)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=15)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=12)
    plt.yticks(tick_marks, classes, fontsize=12)

    fmt = '.2f'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]),range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center", fontsize=17,
                 color="blue" if matrix[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label', fontsize=12)
    plt.xlabel('Predicted label', fontsize=12)
    plt.show()
    
    
def plot_actual_vs_pred(y_true, y_pred, figure=True, show=True):
    """Plots model predictions vs their actual values."""
    if figure:
        plt.figure(figsize=(10, 10))
    
    plt.scatter(y_true, y_pred)
    plt.title('Actual values vs Predicted values.')
    plt.ylabel('Predicted', fontsize=12)
    plt.xlabel('Actual', fontsize=12)
    
    if show:
        plt.show()
    
def plot_roc_curve(y_true, y_pred, figure=True, show=True, label=None):
    """Plots ROC curve."""
    
    threshold = 0.007
    threshold_transformed = stats.boxcox(threshold, 0.25)
    binary_target = y_true > threshold_transformed
    
    fpr, tpr, _ = roc_curve(binary_target, y_pred)
    auc_score = auc(fpr, tpr)
    
    if figure:
        plt.figure(figsize=(10, 10))
    
    temp = 'ROC (AUC = {:.2f})'.format(auc_score)
    label = '{}: {}'.format(label, temp) if label else temp

    plt.plot(fpr, tpr, lw=3, alpha=0.3, label=label)
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    
    if show:
        plt.show()

    
def plot_model_performance(y_true, y_pred, matrix):
    """
    This function plots:
        - predicted values vs actual values
        - confusion matrix
        - roc curve
    """
    print()
    # plotting predicted values vs actual ones
    plot_actual_vs_pred(y_true, y_pred)
              
    print()
    # plotting confusion matrix
    plot_confusion_matrix(matrix, ['Failure', 'Success'])
    
    print()
    # plotting ROC curve
    plot_roc_curve(y_true, y_pred)
    

In [10]:
def scorer(y_true, y_pred):
    """Calculates differnt merics for evaluating our models."""

    threshold = 0.007
    threshold_transformed = stats.boxcox(threshold, 0.25)

    binary_prediction = y_pred > threshold_transformed
    binary_target = y_true > threshold_transformed

    """Regression metrics"""
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    explained_variance = explained_variance_score(y_true, y_pred)
    r_2 = r2_score(y_true, y_pred)

    """Classification metrics"""

    # The probability that the model ranks a random
    # positive example more highly than a random negative example.
    auc = roc_auc_score(binary_target, y_pred)

    conf_matrix = confusion_matrix(binary_target, binary_prediction)
    # true positives (TP): We predicted Y and they do have the disease.
    # true negatives (TN): We predicted N, and they don't have the disease.
    # false positives (FP): We predicted Y, but they don't have the disease.
    # false negatives (FN): We predicted N, but they do have the disease.
    tn, fp, fn, tp = conf_matrix.ravel()

    # correct labels
    total = tn + fp + fn + tp
    actual_yes = fn + tp
    actual_no = total - actual_yes

    # Overall, how often is the classifier correct?
    accuracy = (tp + tn) / total

    # Overall, how often is it wrong?
    misclassification_rate = (fp + fn) / total

    # When it is actually Y, how often does it predict Y?
    # Probability that a test result will be positive when the disease is present.
    # Recall
    tp_rate = tp / actual_yes

    # When it is actually N, how often does it predict N?
    # Probability that a test result will be negative when the disease is not present.
    specificity = tn / actual_no

    # When it is actually N, how often does it predict Y?
    # 1 - specificity
    fp_rate = fp / actual_no

    # When it predicts Y, how often is it correct?
    # Probability that the disease is present when the test is positive.
    # Positive Predictive Value or precision
    pp_value = tp / (fp + tp)

    # When it predicts N, how often is it correct?
    # Probability that the disease is not present when the test is negative.
    # Negative Predictive Value
    np_value = tn / (tn + fn)

    # The weighted average of recall and precision.
    f_score = 2 * tp_rate * pp_value / (tp_rate + pp_value)

    return {
        'rmse': rmse,
        'explained_variance': explained_variance,
        'r_2': r_2,
        'confusion_matrix': conf_matrix,
        'accuracy': accuracy,
        'tpr': tp_rate,
        'specificity': specificity,
        'fpr': fp_rate,
        'ppv': pp_value,
        'npv': np_value,
        'f_score': f_score,
        'auc': auc,
    }



**5 fold CV strategy: we will validate our models and tune hyperparameters by 5 fold CV**

In [ ]:
def cv_mean_score(fold_scores):
    """Calculates model mean score based on list of each folds scores."""
    if not fold_scores:
        return
    keys = list(fold_scores[0].keys())
    data = {k: [x[k] for x in fold_scores] for k in keys if k != 'confusion_matrix'}
    return {k: (np.mean(v), np.std(v)) for k, v in data.items()}   

def cv(model, train_data, train_y, n_folds=5, verbose=True):
    """Helper function for doing cross validation and collecting metrics."""

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    training_metrics, validatin_metrics = [], []
    for i, (train_ind, valid_ind) in enumerate(kf.split(train_data)):
        model_ = clone(model)
        model_.fit(train_data.iloc[train_ind], train_y.iloc[train_ind])

        train_y_pred = model_.predict(train_data.iloc[train_ind])
        t_metrics = scorer(train_y.iloc[train_ind].values, train_y_pred)

        valid_y_pred = model_.predict(train_data.iloc[valid_ind])
        v_metrics = scorer(train_y.iloc[valid_ind].values, valid_y_pred)
        tabular_metrics = [[k, t_metrics[k], v_metrics[k]]
                           for k in t_metrics.keys()
                           if k != 'confusion_matrix']
        
        if verbose:
            print('\nfold: {}\n'.format(i + 1))
            print(tabulate(tabular_metrics,
                           headers=['metric_name', 'training_set', 'validation_set'],
                           tablefmt="fancy_grid",
                           floatfmt=",.3f"))

        training_metrics.append(t_metrics)
        validatin_metrics.append(v_metrics)

    mean_t_score = cv_mean_score(training_metrics)
    mean_v_score = cv_mean_score(validatin_metrics)

    tabular_mean_metrics = [
        [k, mean_t_score[k][0], mean_t_score[k][1], mean_v_score[k][0], mean_v_score[k][1]]
        for k in mean_t_score.keys()]

    tabular_mean_metrics = tabulate(
        tabular_mean_metrics,
        headers=['metric_name', 'train: mean', 'train: std', 'valid: mean', 'valid: std'],
        tablefmt="fancy_grid",
        floatfmt=",.3f")
    return tabular_mean_metrics, mean_t_score, mean_v_score

In [ ]:
def evaluate_model(model, data, y, training=True, plot=True, verbose=True):
    """Evaluates model on train or test data."""
    
    if training:
        # fitting model before prediction
        model.fit(data, y)
    
    # predicition
    y_pred = model.predict(data)
    
    metrics = scorer(y, y_pred)
    if plot:
        plot_model_performance(y, y_pred, metrics.pop('confusion_matrix'))
    else:
        metrics.pop('confusion_matrix')
    if verbose:
        for k, v in metrics.items():
            print('{}: {:.3f}'.format(k, v))

    return model, metrics, y_pred



## Modeling

### Data preparation

**Import librairies**

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb

**Train - Test split: we will keep 15% of data for final testing**

In [ ]:
train, test = train_test_split(training_df, test_size=0.15, shuffle=True)
print(train.shape, test.shape)

In [ ]:
# separating response variable
y_train, X_train = train['Market_Share'], train.drop(columns='Market_Share')
y_test, X_test = test['Market_Share'], test.drop(columns='Market_Share')

### Training & Cross Validation

**Base models and their scores on 5 fold CV:**

As models we have selected 4 models: Lasso regression, Elastic Net, ExtraTress regression and Gradient tree boosting. First two models are more linear than other two. First 2 models are modifications of linear regession with additional constraints on regression coefficients. The last 2 models are more no-linear models based on decision trees.

In [ ]:
all_models, all_models_scores = {}, {}

* **LASSO Regression (Sklearn):**

https://en.wikipedia.org/wiki/Lasso_(statistics)

https://scikit-learn.org/stable/modules/linear_model.html#lasso

This model may be very sensitive to outliers. So we need to made it more robust on them. For that we use the sklearn's **Robustscaler()** method on pipeline.

In [ ]:
model_lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0003, max_iter=1e3, tol=1e-5, random_state=1))
score, _, score_dict = cv(model_lasso, X_train, y_train)

all_models['Lasso'] = model_lasso
all_models_scores['Lasso'] = score_dict
print('\nLasso score:\n')
print(score)

* **ElasticNet Regression (Sklearn):**

https://en.wikipedia.org/wiki/Elastic_net_regularization

https://scikit-learn.org/stable/modules/linear_model.html#elastic-net

again made robust to outliers

In [ ]:
model_enet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=0.8, random_state=3))
score, _, score_dict = cv(model_enet, X_train, y_train)

all_models['ElasticNet'] = model_enet
all_models_scores['ElasticNet'] = score_dict
print('\nElasticNet:\n')
print(score)

* **ExtraTrees Regressor (Sklearn):**

https://www.quora.com/What-is-the-extra-trees-algorithm-in-machine-learning

https://scikit-learn.org/stable/modules/ensemble.html#forest

https://stats.stackexchange.com/questions/44382/mathematics-behind-classification-and-regression-trees


In [ ]:
model_extratree = ExtraTreesRegressor(
    n_estimators=200,
    max_depth=50,
    min_samples_split=2, 
    min_samples_leaf=2,
    min_weight_fraction_leaf=0.0002,
    max_leaf_nodes=None,
    max_features='auto',
    min_impurity_decrease=0.00,
    random_state=32,
    n_jobs=2
)

score, _, score_dict = cv(model_extratree, X_train, y_train)

all_models['ExtraTrees'] = model_extratree
all_models_scores['ExtraTrees'] = score_dict
print('\nExtraTrees:\n')
print(score)

* **GradientTreeBoosting (LightGBM) :**

https://en.wikipedia.org/wiki/Gradient_boosting

https://medium.com/mlreview/gradient-boosting-from-scratch-1e317ae4587d

https://lightgbm.readthedocs.io/en/latest/


In [ ]:
model_lgb = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='rmse',
    n_estimators=5000,
    learning_rate=0.03,
    
    max_depth=6,             # Specify the max depth to which tree will grow. 
    num_leaves=24,           # Number of leaves in one tree

    min_child_weight=11,     # minimal sum hessian in one leaf
    min_data_in_leaf=20,     # Min number of data in one leaf.
    
    subsample=0.8,           # Specifies the fraction of data to be used for each iteration
    subsample_freq=1,
    bagging_seed=9,
    
    colsample_bytree=0.8,    # Specifies the fraction of features to be taken for each iteration
    feature_fraction_seed=7,
    
    min_gain_to_split=0.002, # Min gain to perform splitting
    reg_alpha=0.0,
    reg_lambda=0.0,
    n_jobs=2,
)
score, _, score_dict = cv(model_lgb, X_train, y_train)

all_models['GradientTreeBoosting'] = model_lgb
all_models_scores['GradientTreeBoosting'] = score_dict
print('\nGradientTreeBoosting:\n')
print(score)

**Mean CV scores for each model:**

In [ ]:
model_names=[]
tabular_format = defaultdict(list)
for model_name, model_score_dict in all_models_scores.items():
    model_names.append(model_name)
    for metric_name, metric_score in model_score_dict.items():
        tabular_format[metric_name].append(metric_score[0])


tabular_format = tabulate([[k] + v for k, v in tabular_format.items()], 
                          headers=['Metric'] + model_names, floatfmt=",.3f",  tablefmt="fancy_grid")
print(tabular_format)

Some insights from scores: 

* As expected **Lasso** and **ElasticNet** performance are almost similar and their performance is low compared to DecisionTree models, which is also expected as we investigated that our data mostly is non linear.

* If we compare **GradientTreeBoosting** and **ExtraTrees** then we can notice that their performances are comparable: mean validation RMSE is almost equal and AUC is almost equal. We see some difference for tpr (true positive ratio - When it is actually Y, how often does it predict Y?) and fpr (false positive ratio - When it is actually N, how often does it predict Y?) metrics for given 0.7% success threshold. As we can see both are bigger for GradientTreeBoosting based model **(tpr: 0.413 vs 0.324, fpr: 0.068  vs 0.050)**. So what does it mean? We can assume that GradientTreeBoosting based model learned more from data than ExtraTrees (as it is more complex model than ExtraTrees), so it can differentiate more succesful launches, but together with that the false positive ratio is more by 1.8%, which is not good from bussiness prospective. So GradientTreeBoosting based models pays fpr increase cost (by 1.8%) for having higher ptr by 8.9%. 
* All models have low tpr, which is a result of having imbalanced dataset (only 19% are successes in dataset).


**Training scores on full training data for each model:**

In this section using all training data we fit model on it and afterwards using the same training data make predictions. We compare that predictions with actual data. So if the model fitted properly, it should show very high performance metrics (AUC > 0.95, RMSE < 0.06).

In [ ]:
all_fitted_models = {}
all_models_predictions = {}
for name, model in all_models.items():
    print('\n model: {} train score'.format(name))
    fitted_model, _, y_pred = evaluate_model(model, X_train, y_train, training=True, plot=True)
    all_fitted_models[name] = fitted_model
    all_models_predictions[name] = y_pred
    print()

Lets plot all models ROC curves together.

In [ ]:
threshold = 0.007
threshold_transformed = stats.boxcox(threshold, 0.25)
binary_target = y_train > threshold_transformed

plt.figure(figsize=(15, 10))
for k, y_pred in all_models_predictions.items():
    fpr, tpr, _ = roc_curve(binary_target, y_pred)
    auc_score = auc(fpr, tpr)
    label = '{}: ROC (AUC = {:.3f})'.format(k, auc_score)
    plt.plot(fpr, tpr, lw=1, alpha=0.6, label=label)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.title('ROC curve on train data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

As we can see *"linear"* models **Lasso** and **ElasticNet** are not able to fully fit even on training data. So afterwards we will not consider using them. 

In contrast,  *"non linear"* models fitted with training data almost ideally, even more they have been **overfitted**, that is why our validation scores are not high enough. It is also expected as our data is not big enough.

**Feature importances for all models:**

In [ ]:
feat_imp = pd.Series(all_fitted_models['Lasso']._final_estimator.coef_, index=X_train.columns)
feat_imp.sort_values(inplace=True)
plt.figure(figsize=(10, 7))
plt.barh(feat_imp.index[:15], feat_imp.values[:15])
plt.barh(feat_imp.index[-15:], feat_imp.values[-15:])
plt.title('Lassso')
plt.tight_layout()
plt.show()

In [ ]:
feat_imp = pd.Series(all_fitted_models['ElasticNet']._final_estimator.coef_, index=X_train.columns)
feat_imp.sort_values(inplace=True)
plt.figure(figsize=(10, 7))
plt.barh(feat_imp.index[:15], feat_imp.values[:15])
plt.barh(feat_imp.index[-15:], feat_imp.values[-15:])
plt.title('ElascticNet')
plt.tight_layout()
plt.show()

In [ ]:
feat_imp = pd.Series(all_fitted_models['ExtraTrees'].feature_importances_, index=X_train.columns)
feat_imp.nlargest(30).plot(kind='barh', figsize=(8,10))
plt.title('ExtraTrees')
plt.show()

In [ ]:
feat_imp = pd.Series(all_fitted_models['GradientTreeBoosting'].feature_importances_, index=X_train.columns)
feat_imp.nlargest(30).plot(kind='barh', figsize=(8,10))
plt.title('GradientTreeBoosting')
plt.show()

From graphs we can see that different features are important for dfferent models. If we take into consideration only GradiendBoosting and ExtraTrees then we can see for both models there is a common tendency that **RETAILPACKPRICE** related features are important. This is is quite intuitive: in reality market share depends on price.

### Parameter optimization with GridSearch

Now lets try to make some parameter optimization. We hope that this will help to overcome overfitting (at least partially).

In [ ]:
# we will use AUC as an optimization metric, instead of RMSE
auc_scorer = make_scorer(lambda x, y: scorer(x, y)['auc'], greater_is_better=True)

**GradientTreeBoosting**

https://sites.google.com/view/lauraepp/parameters?authuser=0

In [ ]:
lgb_train = lgb.Dataset(X_train, label=y_train)

**Step 1: Fix learning rate and number of estimators for tuning tree-based parameters**

In [ ]:
params = {
    'learning_rate': 0.03,
    'objective':'rmse',
    'max_depth': 6,
    'num_leaves': 31,
    'min_child_weight': 1,
    'subsample': 0.8,
    'subsample_freq': 1,
    'bagging_seed': 9,
    'colsample_bytree': 0.8,
    'feature_fraction_seed': 7,
    'n_jobs': 2,
}

In [ ]:
cvresult = lgb.cv(params, lgb_train, num_boost_round=5000, nfold=5, stratified=False, 
                  metrics=['rmse'], early_stopping_rounds=100)

In [ ]:
n_esitimators = int(1.1 * len(cvresult['rmse-mean']))
print(n_esitimators)

* learning_rate=0.03
* n_esitimators=850

**Step 2: Tune max_depth and num_leaves**

In [ ]:
params_search_1 = {
    'max_depth': range(3, 8, 1), 
    'num_leaves': range(7, 255, 8)
}
gsearch_1 = GridSearchCV(
    estimator=lgb.LGBMRegressor(
        learning_rate=0.03,
        n_estimators=850,
        max_depth=6,
        num_leaves=31,
        min_child_weight=1,
        subsample=0.8,
        subsample_freq=1,
        bagging_seed=9,
        colsample_bytree=0.8,
        feature_fraction_seed=7,
        n_jobs=2),
    param_grid=params_search_1,
    scoring=auc_scorer,
    n_jobs=2,
    iid=False,
    cv=5)

gsearch_1.fit(X_train, y_train)
gsearch_1.best_params_, gsearch_1.best_score_

* learning_rate=0.03
* n_estimators=850
* max_depth=7
* num_leaves=23

**Step 3: Tune min_child_weight and min_child_samples**

In [ ]:
params_search_2 = {
    'min_child_weight': range(1, 50, 5),
    'min_child_samples': range(1, 30, 3)
}
gsearch_2 = GridSearchCV(
    estimator=lgb.LGBMRegressor(
        learning_rate=0.03,
        n_estimators=850,
        max_depth=6,
        num_leaves=31,
        min_child_weight=1,
        subsample=0.8,
        subsample_freq=1,
        bagging_seed=9,
        colsample_bytree=0.8,
        feature_fraction_seed=7,
        n_jobs=2),
    param_grid=params_search_2,
    scoring=auc_scorer,
    n_jobs=2,
    iid=False,
    cv=5)

gsearch_2.fit(X_train, y_train)
gsearch_2.best_params_, gsearch_2.best_score_

* learning_rate=0.03
* n_estimators=850
* max_depth=7
* num_leaves=23
* min_child_weight=1
* min_child_samples=1

**Step 4: Tune min_split_gain**

In [ ]:
params_search_3 = {
    'min_split_gain': np.linspace(0.001, 0.05, num=25)
}

gsearch_3 = GridSearchCV(
    estimator=lgb.LGBMRegressor(
        learning_rate=0.03,
        n_estimators=850,
        max_depth=7,
        num_leaves=23,
        min_child_weight=1,
        min_child_samples=1,
        subsample=0.8,
        subsample_freq=1,
        bagging_seed=9,
        colsample_bytree=0.8,
        feature_fraction_seed=7,
        min_split_gain=0.0,
        n_jobs=2),
    param_grid=params_search_3,
    scoring=auc_scorer,
    n_jobs=2,
    iid=False,
    cv=5)

gsearch_3.fit(X_train, y_train)
gsearch_3.best_params_, gsearch_3.best_score_

* learning_rate=0.03
* n_estimators=850
* max_depth=7
* num_leaves=23
* min_child_weight=1
* min_child_samples=1
* min_split_gain=0.003

**ExtraTrees**

In [ ]:
params_search_4 = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'log2', 0.8],
    'max_depth': [40, 50, 60],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False],
}

gsearch_4 = GridSearchCV(
    estimator=ExtraTreesRegressor(
    n_estimators=200,
    max_depth=50,
    min_samples_split=2, 
    min_samples_leaf=2,
    min_weight_fraction_leaf=0.0002,
    max_leaf_nodes=None,
    max_features='auto',
    min_impurity_decrease=0.00,
    random_state=32,
    n_jobs=2),
    param_grid=params_search_4,
    scoring=auc_scorer,
    n_jobs=2,
    iid=False,
    cv=5)

gsearch_4.fit(X_train, y_train)
gsearch_4.best_params_, gsearch_4.best_score_

Now lets check our models with new parameters.

In [ ]:
final_lgb = lgb.LGBMRegressor(
    boosting_type='gbdt',objective='rmse', n_estimators=850,
    learning_rate=0.03, max_depth=7, num_leaves=23, min_child_weight=1, min_data_in_leaf=1,    
    subsample=0.8, subsample_freq=1, bagging_seed=9, colsample_bytree=0.8, feature_fraction_seed=7,
    min_gain_to_split=0.003, n_jobs=2
)

score, _, _ = cv(final_lgb, X_train, y_train, verbose=False)
print('\nGradientTreeBoosting:\n')
print(score)

In [ ]:
final_extratree = ExtraTreesRegressor(
    n_estimators=300, max_depth=50, min_samples_split=4,
    min_samples_leaf=1, min_weight_fraction_leaf=0.0002,
    max_leaf_nodes=None, max_features='auto', bootstrap=True,
    min_impurity_decrease=0.00, random_state=32, n_jobs=2)

score, _, _ = cv(final_extratree, X_train, y_train, verbose=False)
print('\nExtraTrees:\n')
print(score)

### Testing

Finally lets test our models.

In [ ]:
# first we have to fit using all train data
final_lgb.fit(X_train, y_train)
final_extratree.fit(X_train, y_train);

In [ ]:
# # testing
_, lgb_metrics, lgb_pred = evaluate_model(final_lgb, X_test, y_test, training=False, plot=False, verbose=False)
_, ext_metrics, ext_pred = evaluate_model(final_extratree, X_test, y_test, training=False, plot=False, verbose=False)

In [ ]:
# results
model_names=[]
tabular_format = defaultdict(list)
for model_name, model_score_dict in [('GradientTreeBoosting', lgb_metrics), ('ExtraTrees', ext_metrics)]:
    model_names.append(model_name)
    for metric_name, metric_score in model_score_dict.items():
        tabular_format[metric_name].append(metric_score)

tabular_format = tabulate([[k] + v for k, v in tabular_format.items()], 
                          headers=['Metric'] + model_names, floatfmt=",.3f",  tablefmt="fancy_grid")

print('Test results:')
print(tabular_format)

In [ ]:
threshold = 0.007
threshold_transformed = stats.boxcox(threshold, 0.25)
binary_target = y_test > threshold_transformed

plt.figure(figsize=(15, 10))
for k, y_pred in [('GradientTreeBoosting', lgb_pred), ('ExtraTrees', ext_pred)]:
    fpr, tpr, _ = roc_curve(binary_target, y_pred)
    auc_score = auc(fpr, tpr)
    label = '{}: ROC (AUC = {:.3f})'.format(k, auc_score)
    plt.plot(fpr, tpr, lw=1, alpha=0.6, label=label)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.title('ROC curve on test data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

Now we will calcualte confidence interval to measure how **confident we are that the true success probability will fall within the range of the interval**.
As an observation we should use ppv - positive predictive value, which shows the porbability of having positive sample (aka success), when our model predicts that sample is positive. We will use normal approximation.

**ppv ~ P(true success | model predicts success)**

In [ ]:
def ppv_conf_interval(ppv, total_p_predicted, confidence):
    alpha = 1 - confidence  # target error rate
    z = stats.norm.ppf(1 - alpha / 2)  # 1-alpha/2 - quantile of a standard normal distribution
    se = z * np.sqrt(ppv * (1 - ppv) / total_p_predicted)  # standard error
    return (ppv - se, ppv + se)

In [ ]:
lgb_total_positives = (lgb_pred > threshold_transformed).sum()
extra_tree_total_positives = (ext_pred > threshold_transformed).sum()

print('Total positive predictions:')
print('GradientTreeBoosting: {}, ExtraTrees: {}'.format(lgb_total_positives, extra_tree_total_positives))

tp_lgb = ((lgb_pred > threshold_transformed) * binary_target).sum()
tp_ext = ((ext_pred > threshold_transformed) * binary_target).sum()
actual_true = sum(binary_target)

print('\nCorrect success predictions:')
print('Actual success: {}, GradientTreeBoosting: {}, ExtraTrees: {}'.format(actual_true, tp_lgb, tp_ext))

In [ ]:
print('95% confidence intervals for ppv:')

lgb_conf_interval = ppv_conf_interval(lgb_metrics['ppv'], lgb_total_positives, 0.95)
extra_tree_conf_interval = ppv_conf_interval(ext_metrics['ppv'], extra_tree_total_positives, 0.95)

print('GradientTreeBoosting: [{:.2f}, {:.2f}]'.format(lgb_conf_interval[0], lgb_conf_interval[1]))
print('ExtraTrees: [{:.2f}, {:.2f}]'.format(extra_tree_conf_interval[0], extra_tree_conf_interval[1]))

**Summary**

* **GradientTreeBoosting** is more powerful model than **ExtraTrees** in terms of **tpr (0.473 vs 0.236): probability of predicting success, when it is true success**. 
* **ExtraTrees** is more powerful model than **GradientTreeBoosting** in terms of **fpr (0.010 vs 0.035): probability of predicting success, when it is true failure**.
* As a result of having small **fpr**, **ExtraTrees** has more **positive predictive power (0.86 vs 0.78): probability of being true success, when model predicted success** than **GradientTreeBoosting**. 
* When **ExtraTrees** says success, then with 0.86 probability and [0.69, 1.04] confidence interval it will be true success.
* When **GradientTreeBoosting** says success, then with 0.78 probability and [0.65, 0.93] confidence interval it will be true success.
* **ExtraTrees** says success 2 times rarely than **GradientTreeBoosting** in case true success.


So what model to select??? 

One of the models is very conservative and almost do not predicts wrong successes, but it classifies a lot of successes as failures.
On the other hand second model in some cases can wrongly classify failure as a success, but it 2 times more correctly classifies successes.

The model selection should be calibrated with business needs. If business do not wants to lose money, with possible failures, then we should select **ExtraTrees**, as it in 99% percentage of true failures, predicts failure, but it predicts successs only in 23.6% cases, when it is true success.
If business wants to make a bit more risky investments, then we should select **GradientTreeBoosting** (96.5% vs 99%, 47.3% vs 23.6%).


As a ML algorithm  **GradientTreeBoosting** is more powerful as it has more **fpr** in case of our imbalanced data (only 19% of whole data is success). So we can say that it learned more from data than **ExtraTrees**. It has quite  higher f_score (0.591 vs 0.371), which is harmonic mean of ppv and sensitivity. This means that as a classificator it does it job much better.


Lets test also these models average model.

In [ ]:
average_pred = (lgb_pred + ext_pred) / 2
average_metrics = scorer(y_test, average_pred)
average_metrics.pop('confusion_matrix')

model_names=[]
tabular_format = defaultdict(list)
for model_name, model_score_dict in [('GradientTreeBoosting', lgb_metrics), ('ExtraTrees', ext_metrics), ('AverageModel', average_metrics )]:
    model_names.append(model_name)
    for metric_name, metric_score in model_score_dict.items():
        tabular_format[metric_name].append(metric_score)

tabular_format = tabulate([[k] + v for k, v in tabular_format.items()], 
                          headers=['Metric'] + model_names, floatfmt=",.3f",  tablefmt="fancy_grid")

print('Test results:')
print(tabular_format)

As we see the results are interesting. This model has **fpr=0.010 and ppv=0.913 and tpr=0.382**. So in case of true failure it performs like **ExtraTrees** (almost the same fpr), but in case of true successes its performance much better and close to **GradientTreeBoosting**. Even more its **positive predictive value** is more than for each single model, which is very very good (0.913 vs  0.788 and 0.867). It has also higher auc.

In [ ]:
average_total_positives = (average_pred > threshold_transformed).sum()
tp_average = ((average_pred > threshold_transformed) * binary_target).sum()
average_conf_interval = ppv_conf_interval(average_metrics['ppv'], average_total_positives, 0.95)

In [ ]:
print('\nCorrect success predictions:')
print('Actual success: {}, GradientTreeBoosting: {}, ExtraTrees: {}, AverageModel: {}'.format(
    actual_true, tp_lgb, tp_ext, tp_average))

In [ ]:
print('95% confidence intervals for ppv:')
print('GradientTreeBoosting: [{:.2f}, {:.2f}]'.format(lgb_conf_interval[0], lgb_conf_interval[1]))
print('ExtraTrees: [{:.2f}, {:.2f}]'.format(extra_tree_conf_interval[0], extra_tree_conf_interval[1]))
print('AverageModel: [{:.2f}, {:.2f}]'.format(average_conf_interval[0], average_conf_interval[1]))

As we can see from bussiness prospective average model is better than each single model: it hase more positive predicive value than **ExtraTrees** and less risky than **GradientTreeBoosting**. So as final model we will take AverageModel for making predictions of our score data.

In [ ]:
threshold = 0.007
threshold_transformed = stats.boxcox(threshold, 0.25)
binary_target = y_test > threshold_transformed

plt.figure(figsize=(15, 10))
for k, y_pred in [('GradientTreeBoosting', lgb_pred), ('ExtraTrees', ext_pred), ('AverageModel', average_pred)]:
    fpr, tpr, _ = roc_curve(binary_target, y_pred)
    auc_score = auc(fpr, tpr)
    label = '{}: ROC (AUC = {:.3f})'.format(k, auc_score)
    plt.plot(fpr, tpr, lw=1, alpha=0.6, label=label)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.title('ROC curve on test data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

Now lets find also optimal threshold for our model based on ROC curve.

In [ ]:
fpr, tpr, thresholds = roc_curve(binary_target, average_pred)

# The optimal cut off would be where tpr is high and fpr is low
# tpr - (1-fpr) is zero or near to zero is the optimal cut off point
i = np.arange(len(tpr))  # index for df
roc = pd.DataFrame({'fpr': fpr,
                    'tpr': tpr,
                    '1-fpr': 1 - fpr,
                    'tf': tpr - (1 - fpr),
                    'thresholds':  thresholds})
optimal_threshold = roc.iloc[(roc.tf - 0).abs().argsort()[:1]]

# Plot tpr vs 1-fpr
plt.figure(figsize=(10, 6))
plt.plot(roc['tpr'], label='tpr')
plt.plot(roc['1-fpr'], color='red', label='1-fpr')
plt.xlabel('1-False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend()
plt.tight_layout()
plt.xticks(ticks=[])
plt.show()

In [ ]:
optimal_threshold

In [ ]:
print('optimal threshold: {:.2f}%'.format(inv_boxcox(optimal_threshold['thresholds'], 0.25).values[0]*100))

### Final Prediction

In [ ]:
# Before final prediction we will use all our data for training
final_lgb = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='rmse',
    n_estimators=850,
    learning_rate=0.03,
    max_depth=7,
    num_leaves=23,
    min_child_weight=1,
    min_data_in_leaf=1,
    subsample=0.8,
    subsample_freq=1,
    bagging_seed=9,
    colsample_bytree=0.8,
    feature_fraction_seed=7,
    min_gain_to_split=0.003,
    n_jobs=2)

final_extratree = ExtraTreesRegressor(
    n_estimators=300,
    max_depth=50,
    min_samples_split=4,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0002,
    max_leaf_nodes=None,
    max_features='auto',
    bootstrap=True,
    min_impurity_decrease=0.00,
    random_state=32,
    n_jobs=2)

In [ ]:
final_lgb.fit(training_df.drop(columns='Market_Share'), training_df['Market_Share'])
final_extratree.fit(training_df.drop(columns='Market_Share'), training_df['Market_Share']);

In [ ]:
feat_imp = pd.Series(final_extratree.feature_importances_, index=X_train.columns)
feat_imp.nlargest(30).plot(kind='barh', figsize=(8,10))
plt.title('ExtraTrees')
plt.show()

In [ ]:
feat_imp = pd.Series(final_lgb.feature_importances_, index=X_train.columns)
feat_imp.nlargest(30).plot(kind='barh', figsize=(8,10))
plt.title('GradientTreeBoosting')
plt.show()

In [ ]:
lgb_final_pred = final_lgb.predict(score_df)
ext_final_pred = final_extratree.predict(score_df)

In [ ]:
average_final_pred = (lgb_final_pred + ext_final_pred) / 2
average_final_pred = inv_boxcox(average_final_pred, 0.25)

In [ ]:
initial_score_df = pd.read_csv('score_data.csv')

In [ ]:
initial_score_df['Market_Share'] = average_final_pred
initial_score_df['Success']  = initial_score_df['Market_Share'] > 0.007

In [ ]:
initial_score_df.to_csv('score_data_predicted.csv')